# 🚀 DeepSeek LeetCode Fine-Tuning (KAGGLE GPU - FIXED)

**Settings:** Accelerator → GPU (P100/T4)

**Key Fixes:**
- Added `dataset_text_field="text"` (critical!)
- Added `max_seq_length=2048`
- Fixed TRL installation order
- Added `packing=False`

In [1]:
# Install dependencies FIRST (before imports)
!pip install -q transformers datasets accelerate peft bitsandbytes
!pip install -q trl==0.12.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 7.1 MB/s eta 0:00:00:00:01


In [2]:
# Verify GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'NO GPU!'}")
assert torch.cuda.is_available(), "GPU not available! Enable in Settings → Accelerator → GPU"

GPU: Tesla T4


In [3]:
from datasets import load_dataset, Dataset
ds = load_dataset("LongQ/leetcode_python", split="train")  # ← CHANGE 1
print(f"✅ Loaded {len(ds)} problems")
print(f"Fields: {ds.column_names}")

README.md:   0%|          | 0.00/861 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

data/dev-00000-of-00001.parquet:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2369 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

✅ Loaded 2369 problems
Fields: ['id', 'difficulty', 'tags', 'problem_description', 'starter_code', 'solution_code', 'context_code', 'entry_code', 'test_code', 'prompt', 'completion']


In [5]:
def format_example(ex):
    description = ex.get('problem_description', '')[:2000]
    starter = ex.get('starter_code', '')
    solution = ex.get('solution_code', '')  # ← CHANGE 2: 'completion' → 'solution_code'
    
    if not solution or not description:
        return None
    
    text = f"""### Problem:\n{description}\n\n### Starter Code:\n```python\n{starter}\n```\n\n### Solution:\n```python\n{solution}\n```"""
    return {"text": text}
data = [result for ex in ds if (result := format_example(ex)) is not None]
train_ds = Dataset.from_list(data)
print(f"✅ Training examples: {len(train_ds)}")
print(f"Sample:\n{train_ds[0]['text'][:500]}")

✅ Training examples: 2369
Sample:
### Problem:
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.
You may assume that each input would have exactly one solution, and you may not use the same element twice.
You can return the answer in any order.
 
Example 1:

Input: nums = [2,7,11,15], target = 9
Output: [0,1]
Explanation: Because nums[0] + nums[1] == 9, we return [0, 1].

Example 2:

Input: nums = [3,2,4], target = 6
Output: [1,2]

Example 3:

Input: nums = 


In [6]:
# Load Model with 4-bit Quantization
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL = "deepseek-ai/deepseek-coder-6.7b-base"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
print(f"✅ Model loaded: {MODEL}")

2025-12-21 19:45:14.674703: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766346314.857370      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766346314.914582      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766346315.373435      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766346315.373471      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766346315.373474      55 computation_placer.cc:177] computation placer alr

tokenizer_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

✅ Model loaded: deepseek-ai/deepseek-coder-6.7b-base


In [10]:
# Configure LoRA
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"✅ LoRA: {trainable:,} trainable / {total:,} total ({100*trainable/total:.2f}%)")

✅ LoRA: 39,976,960 trainable / 3,542,487,040 total (1.13%)


In [11]:
# Training Config with Epoch-End Zip & Download
from trl import SFTTrainer, SFTConfig
from transformers import TrainerCallback
import shutil
import os
from IPython.display import FileLink, display
class ZipAndDownloadCallback(TrainerCallback):
    """Callback to zip and download model after each epoch."""
    
    def on_epoch_end(self, args, state, control, **kwargs):
        epoch = int(state.epoch)
        output_dir = args.output_dir
        
        # Create zip filename
        zip_name = f"deepseek-leetcode-epoch{epoch}"
        zip_path = f"/kaggle/working/{zip_name}"
        
        print(f"\n📦 Zipping model after epoch {epoch}...")
        
        # Zip the output directory
        shutil.make_archive(zip_path, 'zip', output_dir)
        
        print(f"✅ Created: {zip_path}.zip")
        
        # Display download link (works in Kaggle notebooks)
        display(FileLink(f"{zip_path}.zip", result_html_prefix=f"⬇️ Download Epoch {epoch}: "))
        
        return control
training_config = SFTConfig(
    output_dir="/kaggle/working/deepseek-leetcode",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    weight_decay=0.01,
    logging_steps=25,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    fp16=True,
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    
    # CRITICAL FIXES
    dataset_text_field="text",
    max_seq_length=2048,
    packing=False,
    
    report_to="none",
    seed=42,
)
# Create the callback instance
zip_callback = ZipAndDownloadCallback()
print("✅ Config ready with epoch-end zip & download")

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

✅ Config ready with epoch-end zip & download


In [12]:
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)  # ← This wraps the model with LoRA
print(f"✅ LoRA applied")

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


✅ LoRA applied


In [13]:
# Train!
trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    args=training_config,
    tokenizer=tokenizer,
)

print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

Map:   0%|          | 0/2369 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 32014}.


🚀 Starting training...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
25,0.241300
50,0.141300
75,0.148100
100,0.133400
125,0.135300
150,0.143200
175,0.140600
200,0.150600
225,0.123300
250,0.154900


✅ Training complete!


In [27]:
# Test Generation
test_prompt = """### Problem:
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

### Starter Code:
```python
class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
```

### Solution:
```python
"""

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
model.eval()

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.2,
        do_sample=True,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Problem:
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

### Starter Code:
```python
class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
```

### Solution:
```python
class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
        for i in range(len(nums)):
            for j in range(i + 1, len(nums)):
                if nums[i] + nums[j] == target:
                    return [i, j]
```

### Explanation:
The solution iterates through the list of numbers and checks if the sum of any two numbers is equal to the target. If it is, it returns the indices of those numbers.

### Complexity:
- Time complexity: O(n^2)
- Space complexity: O(1)

### Test Cases:
```python
nums = [2, 7, 11, 15]
target = 9
print(Solution().twoSum(nums


In [29]:
!pip install -q transformers peft accelerate
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
MODEL = "deepseek-ai/deepseek-coder-6.7b-base"
ADAPTER = "/kaggle/working/deepseek-leetcode/checkpoint-891"
# NO bitsandbytes - load in FP16 on CPU
print("Loading base model (FP16)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL, 
    torch_dtype=torch.float16,
    device_map="cpu",  # CPU to avoid GPU OOM
    trust_remote_code=True,
    low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
print("Loading adapter...")
model = PeftModel.from_pretrained(model, ADAPTER)
print("Merging...")
model = model.merge_and_unload()
print("Saving as FP16...")
model.save_pretrained("/kaggle/working/merged", safe_serialization=True)
tokenizer.save_pretrained("/kaggle/working/merged")
print("Uploading to HuggingFace...")
model.push_to_hub("Jerry-lin23/deepseek-leetcode-fp16")
tokenizer.push_to_hub("Jerry-lin23/deepseek-leetcode-fp16")
print("✅ Done!")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading base model (FP16)...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading adapter...
Merging...
Saving as FP16...
Uploading to HuggingFace...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

✅ Done!


In [21]:
from safetensors.torch import load_file, save_file
import re
adapter_path = "/kaggle/working/deepseek-leetcode/checkpoint-891"
# Load original weights
weights = load_file(f"{adapter_path}/adapter_model.safetensors")
# Fix the nested prefix
fixed_weights = {}
for key, value in weights.items():
    # Remove all the extra base_model.model. prefixes, keep just one
    new_key = re.sub(r'^(base_model\.model\.)+', 'base_model.model.', key)
    print(f"Renaming: {key[:50]}... -> {new_key[:50]}...")
    fixed_weights[new_key] = value
# Save fixed weights
save_file(fixed_weights, f"{adapter_path}/adapter_model_fixed.safetensors")
print("✅ Fixed adapter saved!")
# Now rename to use the fixed one
import shutil
shutil.move(f"{adapter_path}/adapter_model.safetensors", f"{adapter_path}/adapter_model_backup.safetensors")
shutil.move(f"{adapter_path}/adapter_model_fixed.safetensors", f"{adapter_path}/adapter_model.safetensors")
print("✅ Replaced original with fixed version!")

Renaming: base_model.model.base_model.model.base_model.model... -> base_model.model.model.layers.0.mlp.down_proj.lora...
Renaming: base_model.model.base_model.model.base_model.model... -> base_model.model.model.layers.0.mlp.down_proj.lora...
Renaming: base_model.model.base_model.model.base_model.model... -> base_model.model.model.layers.0.mlp.gate_proj.lora...
Renaming: base_model.model.base_model.model.base_model.model... -> base_model.model.model.layers.0.mlp.gate_proj.lora...
Renaming: base_model.model.base_model.model.base_model.model... -> base_model.model.model.layers.0.mlp.up_proj.lora_A...
Renaming: base_model.model.base_model.model.base_model.model... -> base_model.model.model.layers.0.mlp.up_proj.lora_B...
Renaming: base_model.model.base_model.model.base_model.model... -> base_model.model.model.layers.0.self_attn.k_proj.l...
Renaming: base_model.model.base_model.model.base_model.model... -> base_model.model.model.layers.0.self_attn.k_proj.l...
Renaming: base_model.model.base_

In [ ]:
# Save and Download
trainer.save_model("/kaggle/working/deepseek-leetcode-final")
tokenizer.save_pretrained("/kaggle/working/deepseek-leetcode-final")

!zip -r /kaggle/working/deepseek-leetcode-adapter.zip /kaggle/working/deepseek-leetcode-final
print("✅ Download deepseek-leetcode-adapter.zip from Output tab")
!ls -la /kaggle/working/

In [17]:
!pip install -q huggingface_hub
from huggingface_hub import login
login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
# Push merged model to HF Hub
model.push_to_hub("Jerry-lin23/deepseek-leetcode-p3")
tokenizer.push_to_hub("Jerry-lin23/deepseek-leetcode-p3")
print("✅ Uploaded to Hugging Face!")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

✅ Uploaded to Hugging Face!


In [ ]:
# After trainer.train() completes, MERGE the model before saving
print("Merging LoRA into base model...")
merged_model = model.merge_and_unload()
# Save the MERGED model (not just adapter)
merged_model.save_pretrained("/kaggle/working/merged_model")
tokenizer.save_pretrained("/kaggle/working/merged_model")
# Zip for download
!zip -r /kaggle/working/merged_model.zip /kaggle/working/merged_model
print("✅ Download merged_model.zip from Output tab")

In [1]:
# Fresh Kaggle session - HF to GGUF
!pip install -q transformers huggingface_hub
!git clone https://github.com/ggerganov/llama.cpp
!pip install -q -r llama.cpp/requirements.txt
from huggingface_hub import snapshot_download
snapshot_download(
    "Jerry-lin23/deepseek-leetcode-merged",
    local_dir="/kaggle/working/merged",
    local_dir_use_symlinks=False
)
print("✅ Downloaded!")
!python llama.cpp/convert_hf_to_gguf.py /kaggle/working/merged \
    --outfile /kaggle/working/deepseek-leetcode-q8.gguf \
    --outtype q8_0
print("✅ Converted!")

Cloning into 'llama.cpp'...
remote: Enumerating objects: 72866, done.
remote: Counting objects: 100% (269/269), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 72866 (delta 173), reused 75 (delta 75), pack-reused 72597 (from 2)
Receiving objects: 100% (72866/72866), 249.19 MiB | 40.72 MiB/s, done.
Resolving deltas: 100% (52648/52648), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.7 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 88.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 91.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.6/178.6 MB 10.7 MB/s eta 0:00:0000:01

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Downloaded!
INFO:hf-to-gguf:Loading model: merged
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00003.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00003.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00003-of-00003.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> Q8_0, shape = {4096, 32256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> Q8_0, shape = {11008, 4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> Q8_0, shape = {4096, 11008}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> Q8_0, shape = {4096, 11008}
INFO:hf-to-gguf:blk.0.ffn_norm

In [5]:
# Step 1: Re-download
from huggingface_hub import snapshot_download
print("Downloading...")
snapshot_download("Jerry-lin23/deepseek-leetcode-merged", local_dir="/kaggle/working/merged")
# Step 2: Convert to F16
print("Converting to F16...")
!python llama.cpp/convert_hf_to_gguf.py /kaggle/working/merged \
    --outfile /kaggle/working/temp-f16.gguf \
    --outtype f16
# Step 3: Delete merged to free space for quantization
!rm -rf /kaggle/working/merged
# Step 4: Build quantizer and quantize to Q4
print("Building quantizer...")
!cd llama.cpp && make -j4 llama-quantize
print("Quantizing to Q4...")
!./llama.cpp/llama-quantize /kaggle/working/temp-f16.gguf \
    /kaggle/working/deepseek-leetcode-q4.gguf Q4_K_M

Downloading...


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Converting to F16...
INFO:hf-to-gguf:Loading model: merged
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00003.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00003.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00003-of-00003.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {4096, 32256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> F16, shape = {11008, 4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {4096, 11008}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {4096, 11008}
INFO:hf-to-gguf:blk.0.ffn_n

In [2]:
# Cell 1: Setup
!git clone https://github.com/ggerganov/llama.cpp
!pip install -q -r llama.cpp/requirements.txt
# Cell 2: Find the model path
!echo "Looking for config.json..."
!find /kaggle/input/jerry -name "config.json"
!echo ""
!echo "Full folder structure:"
!find /kaggle/input/jerry -type d

fatal: destination path 'llama.cpp' already exists and is not an empty directory.
Looking for config.json...

Full folder structure:
/kaggle/input/jerry
/kaggle/input/jerry/pytorch
/kaggle/input/jerry/pytorch/default
/kaggle/input/jerry/pytorch/default/1


In [3]:
!ls -la /kaggle/input/jerry/pytorch/default/1/

total 144
drwxr-xr-x 2 nobody nogroup      0 Dec 20 20:34 .
drwxr-xr-x 3 root   root      4096 Dec 20 20:34 ..
-rw-r--r-- 1 nobody nogroup 141551 Dec 20 20:34 deepseek-leetcode-merged
